In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
fund = pd.read_excel('/Users/chenqi/Downloads/icbcintern/fund-1/all-fund.xlsx')

In [3]:
class checkNull(object):
    """
    1、For variables with only NULL values, check for the variable name and the corresponding samples -> list, int，pd.DataFrame
    2、For variables with some NULL values, check for the variable name, counts of Null in %, and corresponding samples. -> pd.DataFrame
    3、For variables with repeating values, check for the variable name, counts of distinct values, and corresponding samples.
    """
    def __init__(self, data: pd.DataFrame):
        self.data = data

    def delAnyNull(self):
        # obtain information (name,number of samples, proportion of missing values) about columns that has values in it 
        null_list = []
        for i in self.data.columns[pd.isnull(self.data).any()].values:
            # delete columns which are empty
            null_list.append(i)
        null_num = len(null_list)
        print(f'There are {null_num} features contain null values')
        for i in range(len(null_list)):
            a = len(self.data[null_list[i]][self.data[null_list[i]].isnull()]) / len(self.data[null_list[i]])
            print(f'missing percentage of {null_list[i]}:', end=' ')
            # transform to a percentage
            print(round(a * 100, 2))
            
    def delAllSame(self):
        # find columns which contain only one value
        same_list = []
        for i in self.data.columns.values:
            # remove empty coulmns
            if i not in self.data.columns[pd.isnull(self.data).all()].values:
                # "if thers is only one same value in that column"
                if len(self.data[i].value_counts()) == 1:
                    same_list.append(i)
        same_num = len(same_list)
        print(f'There are {same_num} features contain only repeated values')
        print('Repeated characters are:\n', same_list)


In [4]:
%%capture
c = checkNull(fund)
# chekcing for missing/null values
c.delAnyNull()

In [5]:
%%capture
# checking for repeated values
c.delAllSame()

In [6]:
#### Preprocessing
# fill out missing values as 0
fund = fund.fillna(0)

#### categorize funds
# bond type
bond = fund.query('type =="bond"')
# stock type
stock = fund.query('type=="stock"') 
# blend type
currency = fund.query('type=="blend"') 


In [7]:
# target value
y = bond['sales(10k)']
# feature values
X = bond.drop(columns=['sales(10k)','fund_income_per10k','7day_annual_return','7day_sales','IOPV','level(y/n)','premium_rate','established_date','overflow_rate','code','name','firm','manager','type','investment_type','subscription_status','investment_type'])

# features to keep after 4 types filtering process
#X1 = stock[['size_yuan(100m)','latest_net_worth','annual_return','daily_return','month_net_worth_growth','6month_net_worth_growth','avg_manager_transactions','yrs_of_establishment']]


In [8]:
#one-hot
X1 = pd.DataFrame(pd.get_dummies(X))
#X1 = stock[['size_yuan(100,)','latest_net_worth','annual_return','daily_return','month_net_worth_growth','6month_net_worth_growth','avg_manager_transactions','yrs_of_establishment']]
X1['sales(10k)'] = bond['sales(10k)']

In [9]:
# correlation analysis
X1.corr()[['sales(10k)']].sort_values('sales(10k)',ascending=False)
X1 = X1.drop(columns=['sales(10k)'])

In [10]:
%%capture
#print(X1.corr())
#sns.heatmap(X1.corr(),vmin=0, vmax=1);
colormap=plt.cm.RdBu
plt.figure(figsize=(38,18))
plt.title(" Correlation of Features", y=1.05, size=15)
sns.heatmap(X1.corr(),cmap=colormap,linecolor='white',linewidths=0.1,vmax=1.0,square=True,annot=True)


In [11]:
%%capture
## Feature dimensionality reduction
#1.correlation analysis 2.LASSO Regression  3. Random Forest 4.XGBoost
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# # #normalize the features
ss = MinMaxScaler()
X_scale = pd.DataFrame(ss.fit_transform(X1))
lasso = LassoCV()    # set penality alpha to be 0.01
lasso.fit(X_scale, y)
# print the weights, select the non-zero ones
lasso_dict = dict()
for i in range(len(X1.columns)):
    lasso_dict[X1.columns[i]] = abs(lasso.coef_[i])
print('Ordered by the importance of the features:')
sorted(lasso_dict.items(), key=lambda x:x[1], reverse=True)


In [12]:
def draw_from_dict(lasso_dict,RANGE, heng=0):
   
    sort=sorted(lasso_dict.items(), key=lambda x:x[1], reverse=True)

    x = []
    y = []
    for d in sort:
        x.append(d[0])
        y.append(d[1])
    if heng == 0:
        plt.bar(x[0:RANGE], y[0:RANGE])
        plt.show()
        return 
    elif heng == 1:
        plt.barh(x[0:RANGE], y[0:RANGE])
        plt.show()
        return 
    else:
        return "heng only equals to 1 or 0！"
    
    


In [13]:
%%capture
draw_from_dict(lasso_dict,15,heng=0)

In [14]:
## divide training and evaluating sets
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_scale,y,test_size=0.1,random_state=0)

In [15]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(x_train,y_train)
y_pred_rf = rf.predict(x_test)
imp_df = pd.DataFrame({'features':X1.columns, 'importance': rf.feature_importances_})
imp_df = imp_df.sort_values('importance',ascending=False)
# Kept those features with impotance over 0.03
keep_columns = imp_df[imp_df['importance'] > 0.03]
#X1 = X1[keep_columns]




In [16]:
%%capture
print('Importance of features based on Random Forest：')
keep_columns


In [17]:
%%capture
sns.distplot(rf.feature_importances_)

In [18]:
%%capture
# preparing the df for visualization
dataframe_obj = pd.DataFrame({"x": X1.columns,"y": rf.feature_importances_})
# draw a scatter plot
# for demonstrating general trend, didn't clarify labels on x-axis
sns.jointplot(x="x", y="y", data=dataframe_obj)

In [19]:
%%capture
# evaluation through MSE,MAE
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

print('RF MSE:',mean_squared_error(y_test,y_pred_rf))
print('RF MAE:',mean_absolute_error(y_test,y_pred_rf))
print('RF R2 score: ',r2_score(y_test,y_pred_rf))

# calculating errors
errors = abs(y_pred_rf - y_test)

# mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

print ('MAPE:',np.mean(mape))


In [20]:
%%capture
# Visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))
n = len(x_test) + 1
plt.scatter(np.arange(1,n),y_test, label="real") #default color for real values
plt.scatter(np.arange(1,n),y_pred_rf,color ='g',label="predict") #green for predict values
plt.legend()
plt.show()

In [21]:
%%capture
# XGBOOST
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(x_train,y_train)
y_pred_xgb = xgb.predict(x_test)

# MSE,MAE
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print('XGBOOST MSE:',mean_squared_error(y_test,y_pred_xgb))
print('XGBOOST MAE:',mean_absolute_error(y_test,y_pred_xgb))
print('XGBOOST R2 score: ',r2_score(y_test,y_pred_xgb))

# calculating errors
errors = abs(y_pred_xgb - y_test)

# mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

print ('MAPE:',np.mean(mape))


In [22]:
%%capture
from xgboost import plot_importance
plot_importance(xgb)
plt.show()

In [23]:
%%capture
## MLP with 3 layers
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=[100], activation='relu', solver='adam', 
                    learning_rate_init=0.01, max_iter=200, verbose=True)
mlp.fit(x_train,y_train)
y_pred_mlp = mlp.predict(x_test)

# MSE,MAE
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print('MLP MSE:',mean_squared_error(y_test,y_pred_mlp))
print('MLP MAE:',mean_absolute_error(y_test,y_pred_mlp))
print('MLP R2 score: ',r2_score(y_test,y_pred_mlp))
# calculating errors
errors = abs(y_pred_mlp - y_test)

# mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

print ('MAPE:',np.mean(mape))


In [24]:
%%capture
#Lasso feature weights
draw_from_dict(lasso_dict,15,heng=0)

In [25]:
%%capture
#Random Forest - feature distribution
sns.jointplot(x=X1.columns, y=rf.feature_importances_)

In [26]:
%%capture
#Random Forest - bar plot- feature weights
sns.distplot(rf.feature_importances_)
